<div style="text-align: center; font-family: 'charter bt pro roman'; color: rgb(0, 65, 75);">
    <h1>
    Auxiliar script for releases, revisions and more
    </h1>
</div>

<div style="text-align: center; font-family: 'charter bt pro roman'; color: rgb(0, 65, 75);">
    <h3>
        Documentation
        <br>
        ____________________
            </br>
    </h3>
</div>

<div style="text-align: center; font-family: 'PT Serif Pro Book'; color: rgb(0, 65, 75); font-size: 16px;">
    Jason Cruz
    <br>
    <a href="mailto:jj.cruza@up.edu.pe" style="color: rgb(0, 153, 123); font-size: 16px;">
        jj.cruza@up.edu.pe
    </a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> This auxiliary script serves to load the main project’s input data into SQL, and to merge (concatenate) both the old and new GDP datasets, along with the release datasets. However, the most significant advancement lies in the creation of cumulative and ongoing revisions data, which is derived from the outputs of the main project scripts. Additionally, this script should be viewed as a non-trivial extension to the remaining project scripts.<div/>

<div style="font-family: Amaya; text-align: left; color: rgb(0, 65, 75); font-size:16px">The following <b>outline is functional</b>. By utilising the provided buttons, users are able to enhance their experience by browsing this script.<div/>

<div id="outilne">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #292929; padding: 10px; font-family: 'PT Serif Pro Book';">
    <h2 style="text-align: left; color: #E0E0E0;">
        Outline
    </h2>
    <br>
    <a href="#libraries" style="color: #94FFD8; font-size: 18px;">
        Libraries</a>
    <br>
    <a href="#setup" style="color: #94FFD8; font-size: 18px;">
        Initial set-up</a>
    <br>
    <a href="#selector" style="color: #94FFD8; font-size: 18px;">
        Select <code>sector</code> and <code>frequency</code></a>
    <br>
    <a href="#1" style="color: #94FFD8; font-size: 18px;">
        1. Loading gdp_percentage_structure</a>
    <br>
    <a href="#2" style="color: #94FFD8; font-size: 18px;">
        2. Load date, year and id data</a>
    <br>
    <a href="#3" style="color: #94FFD8; font-size: 18px;">    
        3. Load date, year, id and base year data</a>
    <br>
    <a href="#4" style="color: #94FFD8; font-size: 18px;">    
        4. Loading the dataset with dummy variable to identify if the old raw data of Table 1 and Table 2 is available</a>
    <br>
    <a href="#5" style="color: #94FFD8; font-size: 18px;">    
        5. Load irregular calendar of revisions</a>
    <br>
    <a href="#6" style="color: #94FFD8; font-size: 18px;">    
        6. Concatenate old_gdp_dataset with new_gdp_dataset</a>
    <br>
    <a href="#7" style="color: #94FFD8; font-size: 18px;">    
        7. Merge all the releases datasets</a>
    <br>
    <a href="#8" style="color: #94FFD8; font-size: 18px;">    
        8. Create nowcast errors</a>
    <br>
    <a href="#9" style="color: #94FFD8; font-size: 18px;">    
        9. Create revisions</a>
    <br>
    <a href="#10" style="color: #94FFD8; font-size: 18px;">    
        10. Merge both nowcast errors and revisions</a>
    <br>
    <a href="#11" style="color: #94FFD8; font-size: 18px;">    
        11. Merge revisions and releases</a>
    <br>
    <a href="#12" style="color: #94FFD8; font-size: 18px;">    
        12. Merge forecast errors and releases</a>
    <br>
    <a href="#13" style="color: #94FFD8; font-size: 18px;">    
        13. Create revision dummies</a>
    <br>
    <a href="#14" style="color: #94FFD8; font-size: 18px;">    
        14. Create base year dummies of revision</a>
</div>


<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Any questions or issues regarding the coding, please <a href="mailto:jj.cruza@alum.up.edu.pe" style="color: rgb(0, 153, 123)">email Jason Cruz
    </a>.
    <div/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px"">
    If you don't have the libraries below, please use the following code (as example) to install the required libraries.
    <div/>

In [ ]:
#!pip install os # Comment this code with "#" if you have already installed this library.

<div id="libraries">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark;">
    <h2>
    Libraries
    </h2>
    <div/>

In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine
import re

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="setup">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark;">
    <h2>
    Initial set-up
    </h2>
    <div/>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> The following code lines will create folders in your current path, call them to import and export your outputs. <div/>

In [2]:
# Folder path to save the csv file of GDP percentage structure

gdp_perc_structure_folder = 'gdp_perc_structure'
if not os.path.exists(gdp_perc_structure_folder):
    os.makedirs(gdp_perc_structure_folder) # to create the folder (if it doesn't exist)

In [3]:
# Folder path to save the csv file of NS dates

ns_dates_folder = 'ns_dates'
if not os.path.exists(ns_dates_folder):
    os.makedirs(ns_dates_folder) # to create the folder (if it doesn't exist)

In [4]:
# Folder path to save the csv file of NS base year

ns_base_year_folder = 'ns_base_year'
if not os.path.exists(ns_base_year_folder):
    os.makedirs(ns_base_year_folder) # to create the folder (if it doesn't exist)

In [5]:
# Folder path to save specific xlsx files of Table 1 and 2 (those extracted by OCR or by hand)

old_raw_data_available_folder = 'old_raw_data_available'
if not os.path.exists(old_raw_data_available_folder):
    os.makedirs(old_raw_data_available_folder) # to create the folder (if it doesn't exist)

In [6]:
# Folder path to save the irregular calendar of revisions (.xlsx) files 

irregular_calendar_folder = 'revisions_irregular_calendar'
if not os.path.exists(irregular_calendar_folder):
    os.makedirs(irregular_calendar_folder) # to create the folder (if it doesn't exist)

<p style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> The following function will establish a connection to the <code>gdp_revisions_datasets</code> database in <code>PostgreSQL</code>. The <b>input data</b> used in this jupyter notebook will be loaded from this <code>PostgreSQL</code> database, and similarly, all <b>output data</b> generated by this jupyter notebook will be stored in that database. Ensure that you set the necessary parameters to access the server once you have obtained the required permissions.<p/>
    
<p style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
To request permissions, please email Jason Cruz <a href="mailto:jj.cruza@alum.up.edu.pe" style="color: rgb(0, 153, 123); text-decoration: none;"> <span style="font-size: 24px;">&#x2709;</span>
    </a>.
<p/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    <span style="font-size: 24px; color: #FFA823; font-weight: bold;">&#9888;</span>
    Enter your user credentials to acces to SQL.
    <div/>

In [7]:
# Get environment variables
user = os.environ.get('CIUP_SQL_USER')
password = os.environ.get('CIUP_SQL_PASS')
host = os.environ.get('CIUP_SQL_HOST')
port = 5432
database = 'gdp_revisions_datasets'

# Check if all environment variables are defined
if not all([host, user, password]):
    raise ValueError("Some environment variables are missing (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

# Create connection string
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Create SQLAlchemy engine
engine = create_engine(connection_string)

<div style="text-align: left;">
    <span style="font-size: 24px; color: rgb(255, 32, 78); font-weight: bold;">&#9888;</span>
    <span style="font-family: PT Serif Pro Book; color: black; font-size: 16px;">
        Import all other functions required by this jupyter notebook.
    </span>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Please, check the script <code>aux_files_to_sql_functions.py</code> which contains all the functions required by this jupyter notebook. The functions there are ordered according to the <a href="#outilne" style="color: #3d30a2;">sections</a> of this jupyter notebok. Altough some functions are included in <code>gdp_revisions_datasets_functions.py</code>.<div/>

In [8]:
    from aux_files_to_sql_functions import *
from gdp_revisions_datasets_functions import *

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="selector">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark;">
    <h2>
    Select <code>variable</code>, <code>sector</code> and <code>frequency</code>
    </h2>
    <div/>

<div id="step-variable">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Select variable</span>
  </div>

In [9]:
# Call the function to show the popup window
variable = show_variable_window()
print("Selected variable:", variable)

Selected variable: r


<div id="step-sector">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Select economic sector</span>
  </div>

In [10]:
# Call the function to show the popup window
sector = show_sector_window()
print("Selected economic sector:", sector)

Selected economic sector: gdp


<div id="step-frequency">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Select frequency</span>
  </div>

In [32]:
# Call the function to show the popup window
frequency = show_frequency_window()
print("Selected frequency:", frequency)

Selected frequency: monthly


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: #2C4E80; font-weight: bold;">
        <a href="#step-6-3" style="color: #2C4E80; text-decoration: none;">⮟</a>
    </span> 
    <a href="#step-6-3" style="color: #2C4E80; text-decoration: none;">Continue with Step <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> in <b>Section 6</b>.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#6" style="color: #3772ff; text-decoration: none;">⮟</a>
    </span> 
    <a href="#6" style="color: #3772ff; text-decoration: none;">Go to <b>Section 6</b>.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: #2C4E80; font-weight: bold;">
        <a href="#step-7-3" style="color: #2C4E80; text-decoration: none;">⮟</a>
    </span> 
    <a href="#step-7-3" style="color: #2C4E80; text-decoration: none;">Continue with Step <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> in <b>Section 7</b>.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#7" style="color: #3772ff; text-decoration: none;">⮟</a>
    </span> 
    <a href="#7" style="color: #3772ff; text-decoration: none;">Go to <b>Section 7</b>.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: #2C4E80; font-weight: bold;">
        <a href="#step-8-3" style="color: #2C4E80; text-decoration: none;">⮟</a>
    </span> 
    <a href="#step-8-3" style="color: #2C4E80; text-decoration: none;">Continue with Step <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> in <b>Section 8</b>.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#8" style="color: #3772ff; text-decoration: none;">⮟</a>
    </span> 
    <a href="#8" style="color: #3772ff; text-decoration: none;">Go to <b>Section 8</b>.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: #2C4E80; font-weight: bold;">
        <a href="#step-9-3" style="color: #2C4E80; text-decoration: none;">⮟</a>
    </span> 
    <a href="#step-9-3" style="color: #2C4E80; text-decoration: none;">Continue with Step <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> in <b>Section 9</b>.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#9" style="color: #3772ff; text-decoration: none;">⮟</a>
    </span> 
    <a href="#9" style="color: #3772ff; text-decoration: none;">Go to <b>Section 9</b>.</a>
</div>

<div id="1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">1.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Loading gdp_percentage_structure</span></h1>
    </div>

In [ ]:
# Loading csv data
gdp_percentage_structure = f'{gdp_perc_structure_folder}/gdp_percentage_structure.csv'

# Reads the CSV file in a DataFrame, specifying that the delimiter is the comma
gdp_percentage_structure = pd.read_csv(gdp_percentage_structure, delimiter=';')
gdp_percentage_structure.head(5)

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Loading to SQL.<div/>

In [ ]:
gdp_percentage_structure.to_sql('gdp_percentage_structure', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">2.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Load date, year and id data</span></h1>
    </div>

In [ ]:
# Loading csv data
csv_file = f'{ns_dates_folder}/dates.csv'

# Reads the CSV file in a DataFrame, specifying that the delimiter is the comma
dates = pd.read_csv(csv_file, delimiter=';')
dates.head(5)

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Clean-up the dataframe columns.<div/>

In [ ]:
# Convert the 'date' column to datetime type without time
dates['date'] = pd.to_datetime(dates['date'], format='%d/%m/%Y', errors='coerce')

# Check the data type of the 'date' column after conversion
print(dates['date'].dtype)

# Check if there are NaT values in the 'date' column
print(dates['date'].isnull().sum())

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Loading to SQL.<div/>

In [ ]:
dates.to_sql('dates_growth_rates', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">3.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Load date, year, id and base year data</span></h1>
    </div>

In [ ]:
# Loading csv data
csv_file = f'{ns_base_year_folder}/base_year.csv'

# Reads the CSV file in a DataFrame, specifying that the delimiter is the comma
base_year = pd.read_csv(csv_file, delimiter=';')
base_year.head(5)

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Clean-up the dataframe columns.<div/>

In [ ]:
# Convert the 'date' column to datetime type without time
base_year['date'] = pd.to_datetime(base_year['date'], format='%d/%m/%Y', errors='coerce')

# Check the data type of the 'date' column after conversion
print(base_year['date'].dtype)

# Check if there are NaT values in the 'date' column
print(base_year['date'].isnull().sum())

In [ ]:
# Change to str
base_year['year'] = base_year['year'].astype(str)
base_year['id_ns'] = base_year['id_ns'].apply(lambda x: f'{x:02}')
base_year['base_year'] = base_year['base_year'].astype(str)
base_year.head(10)

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Loading to SQL.<div/>

In [ ]:
base_year.to_sql('ns_base_year', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">4.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Loading the dataset with dummy variable to identify if the old raw data of Table 1 and Table 2 is available</span></h1>
    </div>

In [ ]:
# Loading csv data
old_raw_data_dummies = f'{old_raw_data_available_folder}/old_raw_data_available.csv'

# Reads the CSV file in a DataFrame, specifying that the delimiter is the comma
old_raw_data_dummies = pd.read_csv(old_raw_data_dummies, delimiter=';')
old_raw_data_dummies.head(5)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    <span style="font-size: 24px; color: #FFA823; font-weight: bold;">&#9888;</span>
        No additional cleaning is required. The original variable type will work perfectly when used in the <code>old_gdp_datasets</code> script.
    <div/>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Loading to SQL.<div/>

In [ ]:
old_raw_data_dummies.to_sql('old_raw_data_available', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">5.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Load irregular calendar of revisions</span></h1>
    </div>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:18px"><b>Loading irregular calendar of revisions for table 1.</b><div/>

In [ ]:
# Loading csv data
irregular_calendar_1 = f'{irregular_calendar_folder}/irregular_calendar_table_1.csv'

# Reads the CSV file in a DataFrame, specifying that the delimiter is the comma
irregular_calendar_1 = pd.read_csv(irregular_calendar_1, delimiter=';')
irregular_calendar_1.head(5)

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Clean-up the dataframe columns.<div/>

In [ ]:
# Change to str
irregular_calendar_1['year'] = irregular_calendar_1['year'].astype(str)
irregular_calendar_1['id_ns'] = irregular_calendar_1['id_ns'].apply(lambda x: f'{x:02}')
irregular_calendar_1['table_number'] = irregular_calendar_1['table_number'].astype(str)
irregular_calendar_1.head(10)

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Loading to SQL.<div/>

In [ ]:
irregular_calendar_1.to_sql(f'revisions_irregular_calendar_table_1', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:18px"><b>Loading irregular calendar of revisions for table 2.</b><div/>

In [ ]:
# Loading csv data
irregular_calendar_2 = f'{irregular_calendar_folder}/irregular_calendar_table_2.csv'

# Reads the CSV file in a DataFrame, specifying that the delimiter is the comma
irregular_calendar_2 = pd.read_csv(irregular_calendar_2, delimiter=';')
irregular_calendar_2.head(5)

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Clean-up the dataframe columns.<div/>

In [ ]:
# Change to str
irregular_calendar_2['year'] = irregular_calendar_2['year'].astype(str)
irregular_calendar_2['id_ns'] = irregular_calendar_2['id_ns'].apply(lambda x: f'{x:02}')
irregular_calendar_2['table_number'] = irregular_calendar_2['table_number'].astype(str)
irregular_calendar_2.head(10)

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Loading to SQL.<div/>

In [ ]:
irregular_calendar_2.to_sql(f'revisions_irregular_calendar_table_2', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:18px"><b>Merging the irregular calendar of Table 1 and Table 2.</b><div/>

In [ ]:
irregular_calendar_1_2 = pd.merge(irregular_calendar_1, irregular_calendar_2, on=['year', 'id_ns'], how='inner')

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px">Keep key columns.<div/>

In [ ]:
irregular_calendar_1_2 = irregular_calendar_1_2[['year', 'id_ns']]
irregular_calendar_1_2

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Clean-up the dataframe columns.<div/>

In [ ]:
# Change to str
irregular_calendar_1_2['year'] = irregular_calendar_1_2['year'].astype(str)
irregular_calendar_1_2['id_ns'] = irregular_calendar_1_2['id_ns'].apply(lambda x: f'{x:02}')
irregular_calendar_1_2.head(10)

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Loading to SQL.<div/>

In [ ]:
irregular_calendar_1_2.to_sql(f'revisions_irregular_calendar_merged', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="6">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">6.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Concatenate old_gdp_dataset with new_gdp_dataset</span></h1>
    </div>

<div id="steps-6">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#step-6-3" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> Import dataframes from PostgresSQL</a>
    <br>
    <a href="#step-6-4" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> Concatenate dataframes</a>
    <br>
    <a href="#step-6-5" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> Load to PostgresSQL</a>
</div>

<div id="step-6-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Import dataframes from PostgresSQL</span>
  </div>

In [ ]:
# Crear un DataFrame de pandas desde la tabla
query_1 = f"SELECT * FROM old_{sector}_{frequency}_growth_rates;"
df_1 = pd.read_sql(query_1, engine)

In [ ]:
# Crear un DataFrame de pandas desde la tabla
query_2 = f"SELECT * FROM new_{sector}_{frequency}_growth_rates;"
df_2 = pd.read_sql(query_2, engine)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-5" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-5" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="step-6-4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> <span>Concatenate dataframes</span>
  </div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Comment the code below if you want the default option (display rows and columns of the dataframe in a limited way)
    <div/>

In [ ]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Run the function to create concatenated dataframe
    <div/>

In [ ]:
def concatenate_dataframes(df1, df2):
    # Concatenate the dataframes
    concatenated_df = pd.concat([df1, df2], ignore_index=True)
    
    return concatenated_df

# Call the function
globals()[f'{sector}_{frequency}_growth_rates'] = concatenate_dataframes(df_1, df_2)
df = globals()[f'{sector}_{frequency}_growth_rates']

# Display the resulting dataframe
df.head(10)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-5" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-5" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="step-6-5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> <span>Load to PostgresSQL</span>
  </div>

In [ ]:
df.to_sql(f'{sector}_{frequency}_growth_rates', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-5" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-5" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-sector" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-sector" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select economic sector</b>.</a>
</div>

In [ ]:
df.columns

In [ ]:
print(df['date'].dtype)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-5" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-5" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="7">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">7.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Merge all the releases datasets</span></h1>
    </div>

<div style="background-color: #0079FF; padding: 6px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <p style="text-align: left; color: #FFFFFF; font-size: 24px; margin: 10;">
        \( r_t(h) \)
    </p>
</div>

<div id="r-steps-7">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#r-step-7-3" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> Import dataframes from PostgresSQL</a>
    <br>
    <a href="#r-step-7-4" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> Merge dataframes</a>
    <br>
    <a href="#r-step-7-5" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> Order by vintages_date</a>
    <br>
    <a href="#r-step-7-6" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10127;</span> Load time-series dataset to PostgresSQL</a>
    <br>
    <a href="#r-step-7-7" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10128;</span> To panel</a>
    <br>
    <a href="#r-step-7-8" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10129;</span> Load panel dataset to PostgresSQL</a>
</div>

<div id="r-step-7-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Import dataframes from PostgresSQL</span>
  </div>

<div style="background-color: #F5F5F5; padding: 6px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <h2 style="text-align: left; color: #222831; font-size: 24px; margin: 10;">
        GDP
    </h2>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select frequency</b>.</a>
</div>

In [12]:
# Create a pandas DataFrame from the SQL table
r_query_releases_gdp = f"SELECT * FROM r_gdp_{frequency}_releases;"
r_query_releases_affected_gdp = f"SELECT * FROM r_gdp_{frequency}_releases_affected;"
r_query_releases_base_year_dummies_gdp = f"SELECT * FROM r_gdp_{frequency}_releases_base_year_dummies;"
r_query_releases_seasonal_dummies_gdp = f"SELECT * FROM r_gdp_{frequency}_releases_seasonal_dummies;"

In [13]:
r_query_releases_gdp_df = pd.read_sql(r_query_releases_gdp, engine)
r_query_releases_affected_gdp_df = pd.read_sql(r_query_releases_affected_gdp, engine)
r_query_releases_base_year_dummies_gdp_df = pd.read_sql(r_query_releases_base_year_dummies_gdp, engine)
r_query_releases_seasonal_dummies_gdp_df = pd.read_sql(r_query_releases_seasonal_dummies_gdp, engine)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#r-steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="background-color: #F5F5F5; padding: 6px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <h2 style="text-align: left; color: #222831; font-size: 24px; margin: 10;">
        Agriculture
    </h2>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select frequency</b>.</a>
</div>

In [14]:
# Create a pandas DataFrame from the SQL table
r_query_releases_agriculture = f"SELECT * FROM r_agriculture_{frequency}_releases;"
r_query_releases_affected_agriculture = f"SELECT * FROM r_agriculture_{frequency}_releases_affected;"
r_query_releases_base_year_dummies_agriculture = f"SELECT * FROM r_agriculture_{frequency}_releases_base_year_dummies;"
r_query_releases_seasonal_dummies_agriculture = f"SELECT * FROM r_agriculture_{frequency}_releases_seasonal_dummies;"

In [15]:
r_query_releases_agriculture_df = pd.read_sql(r_query_releases_agriculture, engine)
r_query_releases_affected_agriculture_df = pd.read_sql(r_query_releases_affected_agriculture, engine)
r_query_releases_base_year_dummies_agriculture_df = pd.read_sql(r_query_releases_base_year_dummies_agriculture, engine)
r_query_releases_seasonal_dummies_agriculture_df = pd.read_sql(r_query_releases_seasonal_dummies_agriculture, engine)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#r-steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="background-color: #F5F5F5; padding: 6px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <h2 style="text-align: left; color: #222831; font-size: 24px; margin: 10;">
        Fishing
    </h2>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select frequency</b>.</a>
</div>

In [16]:
# Create a pandas DataFrame from the SQL table
r_query_releases_fishing = f"SELECT * FROM r_fishing_{frequency}_releases;"
r_query_releases_affected_fishing = f"SELECT * FROM r_fishing_{frequency}_releases_affected;"
r_query_releases_base_year_dummies_fishing = f"SELECT * FROM r_fishing_{frequency}_releases_base_year_dummies;"
r_query_releases_seasonal_dummies_fishing = f"SELECT * FROM r_fishing_{frequency}_releases_seasonal_dummies;"

In [17]:
r_query_releases_fishing_df = pd.read_sql(r_query_releases_fishing, engine)
r_query_releases_affected_fishing_df = pd.read_sql(r_query_releases_affected_fishing, engine)
r_query_releases_base_year_dummies_fishing_df = pd.read_sql(r_query_releases_base_year_dummies_fishing, engine)
r_query_releases_seasonal_dummies_fishing_df = pd.read_sql(r_query_releases_seasonal_dummies_fishing, engine)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#r-steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="background-color: #F5F5F5; padding: 6px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <h2 style="text-align: left; color: #222831; font-size: 24px; margin: 10;">
        Mining
    </h2>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select frequency</b>.</a>
</div>

In [18]:
# Create a pandas DataFrame from the SQL table
r_query_releases_mining = f"SELECT * FROM r_mining_{frequency}_releases;"
r_query_releases_affected_mining = f"SELECT * FROM r_mining_{frequency}_releases_affected;"
r_query_releases_base_year_dummies_mining = f"SELECT * FROM r_mining_{frequency}_releases_base_year_dummies;"
r_query_releases_seasonal_dummies_mining = f"SELECT * FROM r_mining_{frequency}_releases_seasonal_dummies;"

In [19]:
r_query_releases_mining_df = pd.read_sql(r_query_releases_mining, engine)
r_query_releases_affected_mining_df = pd.read_sql(r_query_releases_affected_mining, engine)
r_query_releases_base_year_dummies_mining_df = pd.read_sql(r_query_releases_base_year_dummies_mining, engine)
r_query_releases_seasonal_dummies_mining_df = pd.read_sql(r_query_releases_seasonal_dummies_mining, engine)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#r-steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="background-color: #F5F5F5; padding: 6px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <h2 style="text-align: left; color: #222831; font-size: 24px; margin: 10;">
        Manufacturing
    </h2>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select frequency</b>.</a>
</div>

In [20]:
# Create a pandas DataFrame from the SQL table
r_query_releases_manufacturing = f"SELECT * FROM r_manufacturing_{frequency}_releases;"
r_query_releases_affected_manufacturing = f"SELECT * FROM r_manufacturing_{frequency}_releases_affected;"
r_query_releases_base_year_dummies_manufacturing = f"SELECT * FROM r_manufacturing_{frequency}_releases_base_year_dummies;"
r_query_releases_seasonal_dummies_manufacturing = f"SELECT * FROM r_manufacturing_{frequency}_releases_seasonal_dummies;"

In [21]:
r_query_releases_manufacturing_df = pd.read_sql(r_query_releases_manufacturing, engine)
r_query_releases_affected_manufacturing_df = pd.read_sql(r_query_releases_affected_manufacturing, engine)
r_query_releases_base_year_dummies_manufacturing_df = pd.read_sql(r_query_releases_base_year_dummies_manufacturing, engine)
r_query_releases_seasonal_dummies_manufacturing_df = pd.read_sql(r_query_releases_seasonal_dummies_manufacturing, engine)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#r-steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="background-color: #F5F5F5; padding: 6px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <h2 style="text-align: left; color: #222831; font-size: 24px; margin: 10;">
        Construction
    </h2>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select frequency</b>.</a>
</div>

In [22]:
# Create a pandas DataFrame from the SQL table
r_query_releases_construction = f"SELECT * FROM r_construction_{frequency}_releases;"
r_query_releases_affected_construction = f"SELECT * FROM r_construction_{frequency}_releases_affected;"
r_query_releases_base_year_dummies_construction = f"SELECT * FROM r_construction_{frequency}_releases_base_year_dummies;"
r_query_releases_seasonal_dummies_construction = f"SELECT * FROM r_construction_{frequency}_releases_seasonal_dummies;"

In [23]:
r_query_releases_construction_df = pd.read_sql(r_query_releases_construction, engine)
r_query_releases_affected_construction_df = pd.read_sql(r_query_releases_affected_construction, engine)
r_query_releases_base_year_dummies_construction_df = pd.read_sql(r_query_releases_base_year_dummies_construction, engine)
r_query_releases_seasonal_dummies_construction_df = pd.read_sql(r_query_releases_seasonal_dummies_construction, engine)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#r-steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="background-color: #F5F5F5; padding: 6px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <h2 style="text-align: left; color: #222831; font-size: 24px; margin: 10;">
        Commerce
    </h2>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select frequency</b>.</a>
</div>

In [24]:
# Create a pandas DataFrame from the SQL table
r_query_releases_commerce = f"SELECT * FROM r_commerce_{frequency}_releases;"
r_query_releases_affected_commerce = f"SELECT * FROM r_commerce_{frequency}_releases_affected;"
r_query_releases_base_year_dummies_commerce = f"SELECT * FROM r_commerce_{frequency}_releases_base_year_dummies;"
r_query_releases_seasonal_dummies_commerce = f"SELECT * FROM r_commerce_{frequency}_releases_seasonal_dummies;"

In [25]:
r_query_releases_commerce_df = pd.read_sql(r_query_releases_commerce, engine)
r_query_releases_affected_commerce_df = pd.read_sql(r_query_releases_affected_commerce, engine)
r_query_releases_base_year_dummies_commerce_df = pd.read_sql(r_query_releases_base_year_dummies_commerce, engine)
r_query_releases_seasonal_dummies_commerce_df = pd.read_sql(r_query_releases_seasonal_dummies_commerce, engine)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#r-steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="background-color: #F5F5F5; padding: 6px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <h2 style="text-align: left; color: #222831; font-size: 24px; margin: 10;">
        Electricity
    </h2>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select frequency</b>.</a>
</div>

In [26]:
# Create a pandas DataFrame from the SQL table
r_query_releases_electricity = f"SELECT * FROM r_electricity_{frequency}_releases;"
r_query_releases_affected_electricity = f"SELECT * FROM r_electricity_{frequency}_releases_affected;"
r_query_releases_base_year_dummies_electricity = f"SELECT * FROM r_electricity_{frequency}_releases_base_year_dummies;"
r_query_releases_seasonal_dummies_electricity = f"SELECT * FROM r_electricity_{frequency}_releases_seasonal_dummies;"

In [27]:
r_query_releases_electricity_df = pd.read_sql(r_query_releases_electricity, engine)
r_query_releases_affected_electricity_df = pd.read_sql(r_query_releases_affected_electricity, engine)
r_query_releases_base_year_dummies_electricity_df = pd.read_sql(r_query_releases_base_year_dummies_electricity, engine)
r_query_releases_seasonal_dummies_electricity_df = pd.read_sql(r_query_releases_seasonal_dummies_electricity, engine)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#r-steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="background-color: #F5F5F5; padding: 6px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <h2 style="text-align: left; color: #222831; font-size: 24px; margin: 10;">
        Services
    </h2>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select frequency</b>.</a>
</div>

In [28]:
# Create a pandas DataFrame from the SQL table
r_query_releases_services = f"SELECT * FROM r_services_{frequency}_releases;"
r_query_releases_affected_services = f"SELECT * FROM r_services_{frequency}_releases_affected;"
r_query_releases_base_year_dummies_services = f"SELECT * FROM r_services_{frequency}_releases_base_year_dummies;"
r_query_releases_seasonal_dummies_services = f"SELECT * FROM r_services_{frequency}_releases_seasonal_dummies;"

In [29]:
r_query_releases_services_df = pd.read_sql(r_query_releases_services, engine)
r_query_releases_affected_services_df = pd.read_sql(r_query_releases_affected_services, engine)
r_query_releases_base_year_dummies_services_df = pd.read_sql(r_query_releases_base_year_dummies_services, engine)
r_query_releases_seasonal_dummies_services_df = pd.read_sql(r_query_releases_seasonal_dummies_services, engine)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#r-steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="r-step-7-4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> <span>Merge dataframes</span>
  </div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Run the function to create merged dataframe for releases
    <div/>

In [36]:
merged_releases = releases_datasets_merge(frequency,
                                          r_query_releases_gdp_df,
                                          r_query_releases_agriculture_df,
                                          r_query_releases_fishing_df,
                                          r_query_releases_mining_df,
                                          r_query_releases_manufacturing_df,
                                          r_query_releases_construction_df,
                                          r_query_releases_commerce_df,
                                          r_query_releases_electricity_df,
                                          r_query_releases_services_df)

In [37]:
merged_releases_affected = releases_datasets_merge(frequency,
                                          r_query_releases_affected_gdp_df,
                                          r_query_releases_affected_agriculture_df,
                                          r_query_releases_affected_fishing_df,
                                          r_query_releases_affected_mining_df,
                                          r_query_releases_affected_manufacturing_df,
                                          r_query_releases_affected_construction_df,
                                          r_query_releases_affected_commerce_df,
                                          r_query_releases_affected_electricity_df,
                                          r_query_releases_affected_services_df)

In [38]:
merged_releases_base_year_dummies = releases_datasets_merge(frequency,
                                          r_query_releases_base_year_dummies_gdp_df,
                                          r_query_releases_base_year_dummies_agriculture_df,
                                          r_query_releases_base_year_dummies_fishing_df,
                                          r_query_releases_base_year_dummies_mining_df,
                                          r_query_releases_base_year_dummies_manufacturing_df,
                                          r_query_releases_base_year_dummies_construction_df,
                                          r_query_releases_base_year_dummies_commerce_df,
                                          r_query_releases_base_year_dummies_electricity_df,
                                          r_query_releases_base_year_dummies_services_df)

In [39]:
merged_releases_seasonal_dummies = releases_datasets_merge(frequency,
                                          r_query_releases_seasonal_dummies_gdp_df,
                                          r_query_releases_seasonal_dummies_agriculture_df,
                                          r_query_releases_seasonal_dummies_fishing_df,
                                          r_query_releases_seasonal_dummies_mining_df,
                                          r_query_releases_seasonal_dummies_manufacturing_df,
                                          r_query_releases_seasonal_dummies_construction_df,
                                          r_query_releases_seasonal_dummies_commerce_df,
                                          r_query_releases_seasonal_dummies_electricity_df,
                                          r_query_releases_seasonal_dummies_services_df)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#r-steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select frequency</b>.</a>
</div>

<div id="r-step-7-5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> <span>Order by vintages_date</span>
  </div>

In [40]:
merged_releases = sort_by_date(merged_releases)
merged_releases_affected = sort_by_date(merged_releases_affected)
merged_releases_base_year_dummies = sort_by_date(merged_releases_base_year_dummies)
merged_releases_seasonal_dummies = sort_by_date(merged_releases_seasonal_dummies)

,vintages_date,gdp_release_1,gdp_release_2,gdp_release_3,gdp_release_4,gdp_release_5,gdp_release_6,gdp_release_7,gdp_release_8,gdp_release_9,...,services_release_10,services_release_11,services_release_12,services_release_13,services_release_14,services_release_15,services_release_16,services_release_17,services_release_18,services_most_recent
0,1992-01-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1992-02-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1992-03-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1992-04-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1992-05-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
merged_releases.head(5)

,vintages_date,gdp_release_1,gdp_release_2,gdp_release_3,gdp_release_4,gdp_release_5,gdp_release_6,gdp_release_7,gdp_release_8,gdp_release_9,...,services_release_31,services_release_32,services_release_33,services_release_34,services_release_35,services_release_36,services_release_37,services_release_38,services_release_39,services_most_recent
0,1993-01-01,6.4,6.4,6.4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1994-01-01,13.1,13.1,13.1,13.1,13.1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1995-01-01,7.0,7.0,7.0,7.0,7.0,7.2,7.2,7.2,7.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
3,1996-01-01,2.8,2.8,2.8,2.8,2.6,2.6,2.6,2.6,2.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.2
4,1997-01-01,7.4,7.4,7.4,7.2,7.2,7.2,7.2,7.2,7.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.3


In [42]:
merged_releases_affected.head(5)

,vintages_date,gdp_release_1,gdp_release_2,gdp_release_3,gdp_release_4,gdp_release_5,gdp_release_6,gdp_release_7,gdp_release_8,gdp_release_9,...,services_release_10,services_release_11,services_release_12,services_release_13,services_release_14,services_release_15,services_release_16,services_release_17,services_release_18,services_most_recent
0,1992-01-01,1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1992-02-01,1.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1992-03-01,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1992-04-01,-7.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1992-05-01,-5.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
merged_releases_base_year_dummies.head(5)

,vintages_date,gdp_release_1,gdp_release_2,gdp_release_3,gdp_release_4,gdp_release_5,gdp_release_6,gdp_release_7,gdp_release_8,gdp_release_9,...,services_release_10,services_release_11,services_release_12,services_release_13,services_release_14,services_release_15,services_release_16,services_release_17,services_release_18,services_most_recent
0,1992-01-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1992-02-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1992-03-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1992-04-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1992-05-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
merged_releases_seasonal_dummies.head(5)

,vintages_date,gdp_release_1,gdp_release_2,gdp_release_3,gdp_release_4,gdp_release_5,gdp_release_6,gdp_release_7,gdp_release_8,gdp_release_9,...,services_release_10,services_release_11,services_release_12,services_release_13,services_release_14,services_release_15,services_release_16,services_release_17,services_release_18,services_most_recent
0,1992-01-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1992-02-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1992-03-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1992-04-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1992-05-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#r-steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select frequency</b>.</a>
</div>

<div id="r-step-7-6">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10127;</span> <span>Load time-series dataset to PostgresSQL</span>
  </div>

<div style="text-align: left;">
    <span style="font-size: 24px; color: rgb(255, 32, 78); font-weight: bold;">&#9888;</span>
    <span style="font-family: PT Serif Pro Book; color: black; font-size: 16px;">
        Comment or uncomment depending on whether you want to work with releases of growth rates or releases of revision dummies.
    </span>
</div>

In [ ]:
merged_releases.to_sql(f'r_sectorial_gdp_{frequency}_releases', engine, index=False, if_exists='replace')

In [ ]:
merged_releases_affected.to_sql(f'r_sectorial_gdp_{frequency}_releases_affected', engine, index=False, if_exists='replace')

In [ ]:
merged_releases_base_year_dummies.to_sql(f'sectorial_gdp_{frequency}_releases_base_year_dummies', engine, index=False, if_exists='replace')

In [ ]:
merged_releases_seasonal_dummies.to_sql(f'r_sectorial_gdp_monthly_releases_seasonal_dummies', engine, index=False, if_exists='replace') # quarter and monthly revisions

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#r-steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select frequency</b>.</a>
</div>

<div id="r-step-7-7">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10128;</span> <span>To panel</span>
  </div>

In [ ]:
# Create a pandas DataFrame from the SQL table
query_aux_m = f"SELECT * FROM sectorial_gdp_monthly_releases;"
df_releases_m = pd.read_sql(query_aux_m, engine)
df_releases_m.head(10) # showing the first 10 obs

In [ ]:
# Create a pandas DataFrame from the SQL table
query_aux_q = f"SELECT * FROM sectorial_gdp_quarterly_releases;"
df_releases_q = pd.read_sql(query_aux_q, engine)
df_releases_q.head(10) # showing the first 10 obs

In [ ]:
# Create a pandas DataFrame from the SQL table
query_aux_a = f"SELECT * FROM sectorial_gdp_annual_releases;"
df_releases_a = pd.read_sql(query_aux_a, engine)
df_releases_a.head(10) # showing the first 10 obs

In [ ]:
df_releases_panel_m = releases_convert_to_panel(df_releases_m)
df_releases_panel_m

In [ ]:
df_releases_panel_q = releases_convert_to_panel(df_releases_q)
df_releases_panel_q

In [ ]:
df_releases_panel_a = releases_convert_to_panel(df_releases_a)
df_releases_panel_a

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#r-steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="r-step-7-8">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10129;</span> <span>Load panel dataset to PostgresSQL</span>
  </div>

In [ ]:
df_releases_panel_m.to_sql(f'sectorial_gdp_monthly_releases_panel', engine, index=False, if_exists='replace')

In [ ]:
df_releases_panel_q.to_sql(f'sectorial_gdp_quarterly_releases_panel', engine, index=False, if_exists='replace')

In [ ]:
df_releases_panel_a.to_sql(f'sectorial_gdp_annual_releases_panel', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#r-steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-frequency" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select frequency</b>.</a>
</div>

<div style="background-color: #FF0060; padding: 6px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <p style="text-align: left; color: #FFFFFF; font-size: 24px; margin: 10;">
        \( e_t(h) \)
    </p>
</div>

<div style="background-color: #00DFA2; padding: 6px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <p style="text-align: left; color: #FFFFFF; font-size: 24px; margin: 10;">
        \( z_t(h) \)
    </p>
</div>

<div id="8">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">8.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create nowcast errors</span></h1>
    </div>

<div id="steps-8">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#step-8-3" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> Import dataframes from PostgresSQL</a>
    <br>
    <a href="#step-8-4" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> Compute cumulative revisions</a>
    <br>
    <a href="#step-8-5" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> Keep cumulative revisions</a>
    <br>
    <a href="#step-8-6" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10127;</span> Transpose dataframe in order to have nowcast error (cumulative revisions) of fixed events as columns</a>
</div>

<div id="step-8-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Import dataframes from PostgresSQL</span>
  </div>

<div style="text-align: left;">
    <span style="font-size: 24px; color: rgb(255, 32, 78); font-weight: bold;">&#9888;</span>
    <span style="font-family: PT Serif Pro Book; color: black; font-size: 16px;">
        Comment or uncomment depending on whether you want to work with <b>bench</b> or <b>no-bench</b> dataset.
    </span>
</div>

In [ ]:
# Create a pandas DataFrame from the SQL table
#query_releases_1 = f"SELECT * FROM sectorial_gdp_{frequency}_releases;"
query_releases_1 = f"SELECT * FROM sectorial_gdp_{frequency}_releases_bench;"
df_releases_1 = pd.read_sql(query_releases_1, engine)
df_releases_1.head(10) # showing the first 10 obs

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Filter releases dataset
    <div/>

In [ ]:
# Set dataframe inn date range
df_releases_filtered_1 = df_releases_1[(df_releases_1['vintages_date'] >= '1993-01-01') & (df_releases_1['vintages_date'] <= '2023-12-31')]

# Order the dataframe by the column 'vintages_date'
df_releases_filtered_1 = df_releases_filtered_1.sort_values(by='vintages_date')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="step-8-4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> <span>Compute cumulative revisions</span>
  </div>

In [ ]:
# Apply the function to your dataframe and get max_release
df_rev_cum, max_release = calculate_cumulative_revisions(df_releases_filtered_1)

# Output the max_release for verification
print(f"The maximum release number found: {max_release}")

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    <span style="font-size: 24px; color: rgb(255, 32, 78); font-weight: bold;">&#9888;</span>
    Computing specific cumulative revisions
    <div/>

In [ ]:
#df_rev_specific_cum = calculate_specific_cumulative_revisions(df_rev_cum, frequency)
#df_rev_specific_cum

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Inspect specific columns
    <div/>

In [ ]:
df_rev_cum.columns

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To SQL
    <div/>

In [ ]:
#df_rev_cum.to_sql(f'sectorial_gdp_{frequency}_cum_revisions_releases', engine, index=False, if_exists='replace')
df_rev_cum.to_sql(f'sectorial_gdp_{frequency}_cum_revisions_releases_bench', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="step-8-5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> <span>Keep cumulative revisions</span>
  </div>

In [ ]:
df_rev_cum.head(10)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Keeping cumulative revisions and date column only 
    <div/>

In [ ]:
df_rev_cum = keep_nowcast_errors(df_rev_cum, frequency) # this is optional in order to compress dataframe columns

In [ ]:
df_rev_cum.columns

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To SQL
    <div/>

In [ ]:
#df_rev_cum.to_sql(f'sectorial_gdp_{frequency}_cum_revisions', engine, index=False, if_exists='replace')
df_rev_cum.to_sql(f'sectorial_gdp_{frequency}_cum_revisions_bench', engine, index=False, if_exists='replace')

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To data panel
    <div/>

In [ ]:
df_rev_cum.head(5)

In [ ]:
df_rev_cum_panel = cum_convert_to_panel(df_rev_cum)

In [ ]:
df_rev_cum_panel

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To SQL
    <div/>

In [ ]:
#df_rev_cum_panel.to_sql(f'sectorial_gdp_{frequency}_cum_revisions_panel', engine, index=False, if_exists='replace')
df_rev_cum_panel.to_sql(f'sectorial_gdp_{frequency}_cum_revisions_panel_bench', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="9">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">9.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create revisions</span></h1>
    </div>

<div id="steps-9">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#step-9-3" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> Import dataframes from PostgresSQL</a>
    <br>
    <a href="#step-9-4" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> Compute intermediate revisions</a>
    <br>
    <a href="#step-9-5" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> Keep intermediate revisions</a>
    <br>
    <a href="#step-9-6" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10127;</span> Transpose dataframe in order to have nowcast error (intermediate revisions) of fixed events as columns</a>
</div>

<div id="step-9-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Import dataframes from PostgresSQL</span>
  </div>

<div style="text-align: left;">
    <span style="font-size: 24px; color: rgb(255, 32, 78); font-weight: bold;">&#9888;</span>
    <span style="font-family: PT Serif Pro Book; color: black; font-size: 16px;">
        Comment or uncomment depending on whether you want to work with <b>bench</b> or <b>no-bench</b> dataset.
    </span>
</div>

In [ ]:
# Create a pandas DataFrame from the SQL table
#query_releases_2 = f"SELECT * FROM sectorial_gdp_{frequency}_releases;"
query_releases_2 = f"SELECT * FROM sectorial_gdp_{frequency}_releases_bench;"
df_releases_2 = pd.read_sql(query_releases_2, engine)
df_releases_2.head(10) # showing the first 10 obs

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Filter releases dataset
    <div/>

In [ ]:
# Set dataframe inn date range
df_releases_filtered_2 = df_releases_2[(df_releases_2['vintages_date'] >= '1993-01-01') & (df_releases_2['vintages_date'] <= '2023-12-31')]

# Order the dataframe by the column 'vintages_date'
df_releases_filtered_2 = df_releases_filtered_2.sort_values(by='vintages_date')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="step-9-4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> <span>Compute intermediate revisions</span>
  </div>

In [ ]:
# Apply the function to your dataframe and get max_release
df_rev_int, max_release = calculate_intermediate_revisions(df_releases_filtered_2)

# Output the max_release for verification
print(f"The maximum release number found: {max_release}")

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Computing specific intermediate revisions
    <div/>

In [ ]:
#df_rev_int = calculate_specific_intermediate_revisions(df_rev_int, frequency)
#df_rev_int

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Inspect specific columns
    <div/>

In [ ]:
df_rev_int[['vintages_date', 'agriculture_release_1', 'agriculture_release_2', 'r_2_agriculture']].head(5)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To SQL
    <div/>

In [ ]:
#df_rev_int.to_sql(f'sectorial_gdp_{frequency}_int_revisions_releases', engine, index=False, if_exists='replace')
df_rev_int.to_sql(f'sectorial_gdp_{frequency}_int_revisions_releases_bench', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="step-9-5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> <span>Keep intermediate revisions</span>
  </div>

In [ ]:
df_rev_int.head(10)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Keeping intermediate revisions and date column only 
    <div/>

In [ ]:
df_rev_int = keep_revisions(df_rev_int, frequency) # this is optional in order to compress dataframe columns

In [ ]:
df_rev_int.columns

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To SQL
    <div/>

In [ ]:
#df_rev_int.to_sql(f'sectorial_gdp_{frequency}_int_revisions', engine, index=False, if_exists='replace')
df_rev_int.to_sql(f'sectorial_gdp_{frequency}_int_revisions_bench', engine, index=False, if_exists='replace')

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To data panel
    <div/>

In [ ]:
df_rev_int_panel = int_convert_to_panel(df_rev_int)
df_rev_int_panel

In [ ]:
df_rev_int_panel['vintages_date'].dtype

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To SQL
    <div/>

In [ ]:
#df_rev_int_panel.to_sql(f'sectorial_gdp_{frequency}_int_revisions_panel', engine, index=False, if_exists='replace')
df_rev_int_panel.to_sql(f'sectorial_gdp_{frequency}_int_revisions_panel_bench', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="10">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">10.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Merge both nowcast errors and revisions</span></h1>
    </div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Merge
    <div/>

In [ ]:
df_rev_panel = pd.merge(df_rev_cum_panel, df_rev_int_panel, on=['vintages_date', 'horizon'], how='inner')

In [ ]:
df_rev_panel

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To SQL
    <div/>

In [ ]:
df_rev_panel.to_sql(f'sectorial_gdp_{frequency}_revisions_panel', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="11">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">11.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Merge revisions and releases</span></h1>
    </div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Import from SQL
    <div/>

In [ ]:
# Create a pandas DataFrame from the SQL table
query_rel_panel = f"SELECT * FROM sectorial_gdp_{frequency}_releases_panel;"
df_rel_panel = pd.read_sql(query_rel_panel, engine)
df_rel_panel.head(10) # showing the first 10 obs

In [ ]:
# Create a pandas DataFrame from the SQL table
query_rev_panel = f"SELECT * FROM sectorial_gdp_{frequency}_int_revisions_panel;"
df_rev_panel = pd.read_sql(query_rev_panel, engine)
df_rev_panel.head(10) # showing the first 10 obs

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Merge
    <div/>

In [ ]:
df_releases_revisions_panel = pd.merge(df_rel_panel, df_rev_panel, on=['vintages_date', 'horizon'], how='inner')

In [ ]:
df_releases_revisions_panel

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To SQL
    <div/>

In [ ]:
df_releases_revisions_panel.to_sql(f'sectorial_gdp_{frequency}_releases_revisions_panel', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="12">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">12.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Merge forecast errors and releases</span></h1>
    </div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Import from SQL
    <div/>

In [ ]:
# Create a pandas DataFrame from the SQL table
query_rel_panel = f"SELECT * FROM sectorial_gdp_{frequency}_releases_panel;"
df_rel_panel = pd.read_sql(query_rel_panel, engine)
df_rel_panel.head(10) # showing the first 10 obs

In [ ]:
# Create a pandas DataFrame from the SQL table
query_cum_panel = f"SELECT * FROM sectorial_gdp_{frequency}_cum_revisions_panel;"
df_cum_panel = pd.read_sql(query_cum_panel, engine)
df_cum_panel.head(10) # showing the first 10 obs

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Merge
    <div/>

In [ ]:
df_releases_errors_panel = pd.merge(df_rel_panel, df_cum_panel, on=['vintages_date', 'horizon'], how='inner')

In [ ]:
df_releases_errors_panel

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To SQL
    <div/>

In [ ]:
df_releases_errors_panel.to_sql(f'sectorial_gdp_{frequency}_releases_errors_panel', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="13">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">13.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create revision dummies</span></h1>
    </div>

<div id="steps-13">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#step-13-3" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> Import dataframes from PostgresSQL</a>
    <br>
    <a href="#step-13-4" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> Compute intermediate revisions</a>
    <br>
    <a href="#step-13-5" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> Keep intermediate revisions</a>
    <br>
    <a href="#step-13-6" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10127;</span> Transpose dataframe in order to have nowcast error (intermediate revisions) of fixed events as columns</a>
</div>

<div id="step-13-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Import dataframes from PostgresSQL</span>
  </div>

In [ ]:
# Create a pandas DataFrame from the SQL table
query_releases_dummies_2 = f"SELECT * FROM sectorial_gdp_monthly_releases_dummies;"
df_releases_dummies_2 = pd.read_sql(query_releases_dummies_2, engine)
df_releases_dummies_2.head(10) # showing the first 10 obs

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Filter releases dataset
    <div/>

In [ ]:
# Set dataframe inn date range
df_releases_dummies_filtered_2 = df_releases_dummies_2[(df_releases_dummies_2['vintages_date'] >= '1993-01-01') & (df_releases_dummies_2['vintages_date'] <= '2023-12-31')]

# Order the dataframe by the column 'vintages_date'
df_releases_dummies_filtered_2 = df_releases_dummies_filtered_2.sort_values(by='vintages_date')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="step-13-4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> <span>Compute intermediate revisions</span>
  </div>

In [ ]:
# Apply the function to your dataframe and get max_release
df_rev_int_dummies, max_release_dummies = calculate_intermediate_revisions_dummies(df_releases_dummies_filtered_2)

# Output the max_release for verification
print(f"The maximum release number found: {max_release_dummies}")

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Computing specific intermediate revisions
    <div/>

In [ ]:
#df_rev_int = calculate_specific_intermediate_revisions(df_rev_int, frequency)
#df_rev_int

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Inspect specific columns
    <div/>

In [ ]:
df_rev_int_dummies[['vintages_date', 'agriculture_release_1', 'agriculture_release_2', 'r_2_agriculture']]

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="step-13-5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> <span>Keep intermediate revisions</span>
  </div>

In [ ]:
df_rev_int_dummies.head(10)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Keeping intermediate revisions and date column only 
    <div/>

In [ ]:
df_rev_int_dummies = keep_revisions(df_rev_int_dummies, frequency) # this is optional in order to compress dataframe columns

In [ ]:
df_rev_int_dummies.columns

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To SQL
    <div/>

In [ ]:
#df_rev_int_dummies.to_sql(f'sectorial_gdp_monthly_int_revisions_dummies', engine, index=False, if_exists='replace')

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To data panel
    <div/>

In [ ]:
df_rev_int_panel_dummies = dummies_int_convert_to_panel(df_rev_int_dummies)
df_rev_int_panel_dummies

In [ ]:
df_rev_int_panel_dummies['vintages_date'].dtype

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To SQL
    <div/>

In [ ]:
df_rev_int_panel_dummies.to_sql(f'sectorial_gdp_monthly_int_revisions_panel_dummies', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="14">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">14.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create base year dummies of revisions</span></h1>
    </div>

<div id="steps-14">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#step-14-3" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> Import dataframes from PostgresSQL</a>
    <br>
    <a href="#step-14-4" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> Compute intermediate revisions</a>
    <br>
    <a href="#step-14-5" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> Keep intermediate revisions</a>
    <br>
    <a href="#step-14-6" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10127;</span> Transpose dataframe in order to have nowcast error (intermediate revisions) of fixed events as columns</a>
</div>

<div id="step-14-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Import dataframes from PostgresSQL</span>
  </div>

In [ ]:
# Create a pandas DataFrame from the SQL table
query_releases_dummies_by = f"SELECT * FROM sectorial_gdp_{frequency}_releases_dummies_base_year;"
df_releases_dummies_by = pd.read_sql(query_releases_dummies_by, engine)
df_releases_dummies_by.head(10) # showing the first 10 obs

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Filter releases dataset
    <div/>

In [ ]:
# Set dataframe inn date range
df_releases_dummies_by_filtered = df_releases_dummies_by[(df_releases_dummies_by['vintages_date'] >= '1993-01-01') & (df_releases_dummies_by['vintages_date'] <= '2023-12-31')]

# Order the dataframe by the column 'vintages_date'
df_releases_dummies_by_filtered = df_releases_dummies_by.sort_values(by='vintages_date')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="step-14-4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> <span>Compute intermediate revisions</span>
  </div>

In [ ]:
# Apply the function to your dataframe and get max_release
df_rev_int_dummies_by, max_release_dummies_by = calculate_intermediate_revisions_dummies(df_releases_dummies_by_filtered)

# Output the max_release for verification
print(f"The maximum release number found: {max_release_dummies_by}")

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Computing specific intermediate revisions
    <div/>

In [ ]:
#df_rev_int = calculate_specific_intermediate_revisions(df_rev_int, frequency)
#df_rev_int

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Inspect specific columns
    <div/>

In [ ]:
df_rev_int_dummies_by[['vintages_date', 'agriculture_release_1', 'agriculture_release_2', 'r_2_agriculture']]

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div id="step-14-5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> <span>Keep intermediate revisions</span>
  </div>

In [ ]:
df_rev_int_dummies_by.head(10)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Keeping intermediate revisions and date column only 
    <div/>

In [ ]:
df_rev_int_dummies_by = keep_revisions(df_rev_int_dummies_by, frequency) # this is optional in order to compress dataframe columns

In [ ]:
df_rev_int_dummies_by.columns

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To SQL
    <div/>

In [ ]:
#df_rev_int_dummies.to_sql(f'sectorial_gdp_monthly_int_revisions_dummies', engine, index=False, if_exists='replace')

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To data panel
    <div/>

In [ ]:
df_rev_int_panel_dummies_by = dummies_int_convert_to_panel(df_rev_int_dummies_by)
df_rev_int_panel_dummies_by

In [ ]:
df_rev_int_panel_dummies_by['vintages_date'].dtype

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    To SQL
    <div/>

In [ ]:
df_rev_int_panel_dummies_by.to_sql(f'sectorial_gdp_{frequency}_int_revisions_panel_dummies_base_year', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-7" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>